In [2]:
import gym
import numpy as np
import time
from IPython import display

In [3]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [5]:
env.observation_space.n

16

In [6]:
env.action_space.n

4

In [7]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [8]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 5)

In [9]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1)

(0.0, 3)

In [10]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, False)

(0.0, 4)

In [11]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, False)

(1.0, 65)

In [12]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [13]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 57/1000
Average number of steps: 10.982456140350877


In [15]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 122/1000
Average number of steps: 16.049180327868854


In [16]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 734/1000
Average number of steps: 38.80926430517711


In [17]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [18]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [20]:
np.all(v_values_1 >= v_values_0)

True

In [21]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [22]:
np.all(v_values_2 >= v_values_1)

True

In [23]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [24]:
np.all(v_values_3 >= v_values_2)

True

In [25]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [26]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [27]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [28]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [29]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

<ipython-input-28-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [30]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 728/1000
Average number of steps: 37.41483516483517


In [31]:
play_multiple_times(env, policy_1, 1000)

Number of successes: 56/1000
Average number of steps: 11.946428571428571


#POLICY ITERATION

In [46]:
def evaluate_policy(env, policy, gamma=0.9, max_iters=500):
    # Khởi tạo giá trị của tất cả các trạng thái là 0
    values = np.zeros(env.observation_space.n)

    for _ in range(max_iters):
        prev_values = np.copy(values)

        # Cập nhật giá trị của mỗi trạng thái
        for state in range(env.observation_space.n):
            action = policy[state]
            q_value = 0

            # Tính giá trị q cho từng hành động trong mỗi trạng thái
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_values[next_state])

            values[state] = q_value

        # Kiểm tra sự hội tụ
        if np.all(np.isclose(values, prev_values)):
            break

    return values

In [47]:
def policy_improvement(env, v_values, policy, gamma=0.9):
    # Khởi tạo
    v_values_new = np.copy(v_values)
    policy_new = np.copy(policy)
    num_iterations = 0

    # Lặp qua mỗi trạng thái trong môi trường
    for i in range(1000):
        policy_temp = np.copy(policy_new)
        
        for state in range(env.observation_space.n):
            q_values = []

            # Lặp qua mỗi hành động
            for action in range(env.action_space.n):
                q_value = 0

                # Lặp qua mỗi kết quả có thể xảy ra
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * v_values_new[next_state])

                q_values.append(q_value)

            # Chọn hành động tốt nhất và cập nhật v_values_new[state]
            best_action = np.argmax(q_values)
            policy_temp[state] = best_action
            v_values_new[state] = np.max(q_values)

        # Trả về policy mới nếu policy không thay đổi
        if np.all(policy_new == policy_temp):
            break

        # Cập nhật policy nếu có thay đổi
        policy_new = np.copy(policy_temp)

    return policy_new


In [48]:
optimal_policy_FrozenLake_v0 = np.random.randint(0,3,env.observation_space.n)

##FrozenLake-v1


In [58]:
# Policy Iteration - FrozenLake-v1
env = gym.make('FrozenLake-v1')
start_time = time.time()
# Khởi tạo chính sách ngẫu nhiên
policy = np.random.randint(0,3, env.observation_space.n)
# Đánh giá giá trị của chính sách hiện tại
values = evaluate_policy(env, policy)
# Cải thiện chính sách
optimal_policy_FrozenLake_v0 = policy_improvement(env, values, policy,gamma=0.9)
end_time = time.time()

In [59]:
# Value Iteration frozenlake-v1
start_time_vi =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_FrozenLake_v0_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
end_time_vi =time.time()

Converged at 79-th iteration.


<ipython-input-28-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [60]:
play_multiple_times(env, optimal_policy_FrozenLake_v0 , 1000)
play_multiple_times(env, optimal_policy_FrozenLake_v0_2, 1000)

Number of successes: 424/1000
Average number of steps: 28.830188679245282
Number of successes: 760/1000
Average number of steps: 36.39605263157895


##FrozenLack8x8-v1

In [61]:
# Policy Iteration - FrozenLake-v1
env = gym.make('FrozenLake8x8-v1')
start_time8x8 = time.time()
# Khởi tạo chính sách ngẫu nhiên
policy8x8 = np.random.randint(0,3, env.observation_space.n)
# Đánh giá giá trị của chính sách hiện tại
values8x8 = evaluate_policy(env, policy8x8)
# Cải thiện chính sách
optimal_policy_FrozenLake8x8_v0 = policy_improvement(env, values8x8, policy8x8,gamma=0.9)
end_time8x8 = time.time()

In [62]:
# Value Iteration frozenlake-v1
start_time_vi8x8 =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_FrozenLake8x8_v0_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
end_time_vi8x8 =time.time()

Converged at 117-th iteration.


<ipython-input-28-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [63]:
play_multiple_times(env, optimal_policy_FrozenLake8x8_v0, 1000)
play_multiple_times(env, optimal_policy_FrozenLake8x8_v0_2, 1000)

Number of successes: 598/1000
Average number of steps: 69.876254180602
Number of successes: 728/1000
Average number of steps: 73.52747252747253


##Taxi-v3

In [64]:
# Policy Iteration taxi-v3
env = gym.make('Taxi-v3')
start_time_taxi =time.time()
policy_Taxi_v3 = np.random.randint(0,3,env.observation_space.n)
v_values_Taxi_v3 = evaluate_policy(env, policy_Taxi_v3)
optimal_policy_Taxi_v3 = policy_improvement(env, v_values_Taxi_v3,policy_Taxi_v3, gamma=0.9)
end_time_taxi =time.time()

# Value Iteration taxi-v3
start_time_vi_taxi =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_Taxi_v3_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
end_time_vi_taxi =time.time()

Converged at 116-th iteration.


<ipython-input-28-9e926206ddb9>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [65]:
play_multiple_times(env, optimal_policy_Taxi_v3, 1000)
play_multiple_times(env, optimal_policy_Taxi_v3_2, 1000)

Number of successes: 1000/1000
Average number of steps: 13.11
Number of successes: 1000/1000
Average number of steps: 13.171


# So sánh Policy Iteration & Value Iteration

Độ phức tạp:

* Value Iteration có độ phức tạp thời gian cao hơn Policy Iteration trong mỗi vòng lặp do phải duyệt qua tất cả các hành động tại mỗi trạng thái. Độ phức tạp của Value Iteration là O(S^2 * A), trong đó S là số lượng trạng thái và A là số lượng hành động.
* Policy Iteration có độ phức tạp thời gian thấp hơn Value Iteration trong mỗi vòng lặp, vì chỉ tính toán giá trị dựa trên một random policy và sau đó cải thiện policy. Độ phức tạp của Policy Iteration là O(S^2), trong đó S là số lượng trạng thái.

In [66]:
print("Value Iteration - frozenlake-v1 :",end_time-start_time)
print("Policy Iteration - frozenlake-v1:",end_time_vi-start_time_vi)
print("Value Iteration - frozenlake8x8-v1 :",end_time8x8-start_time8x8)
print("Policy Iteration - frozenlake8x8-v1:",end_time_vi8x8-start_time_vi8x8)
print("Value Iteration - taxi-v3 :",end_time_taxi-start_time_taxi)
print("Policy Iteration - taxi-v3:",end_time_vi_taxi-start_time_vi_taxi)

Value Iteration - frozenlake-v0 : 0.010804176330566406
Policy Iteration - frozenlake-v0: 0.06967616081237793
Value Iteration - frozenlake8x8-v0 : 0.04332709312438965
Policy Iteration - frozenlake8x8-v0: 0.3626523017883301
Value Iteration - taxi-v3 : 0.4876139163970947
Policy Iteration - taxi-v3: 2.326550245285034


Lý thuyết cho thấy PI có độ phức tạp thời gian cho từng vòng lặp nhỏ hơn so với VI. Tuy nhiên, kết quả thực nghiệm cho thấy PI mất nhiều thời gian hơn để cho ra kết quả so với VI.
* VI nhanh hơn so với PI rất nhiều ( gấp 4 - 8 lần )
* Ở Taxi-v3 PI thực sự mất nhiều thời gian ( 2.32655... )


Có thể suy ra rằng, việc đánh giá và cải thiện chiến lược cho từng lần lặp của PI tốn nhiều thời gian hơn so với quá trình tìm giá trị tối ưu trong VI, dẫn đến PI mất nhiều thời gian hơn trong thực nghiệm.